In [1]:
## Notebook settings
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
## fastai imports
import random
from IPython.core.debugger import set_trace
from fastai.basics import *
from fastai.data_block import ItemList
from fastai.vision import *
from exp.nb_AudioCommon import *
from exp.nb_DataBlock import *
from exp.nb_DataAugmentation import *

In [3]:
def make_learner(data):
    newlayer = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    newlayer = newlayer.cuda()
    learn = cnn_learner(data, models.resnet18, metrics=accuracy)
    learn.model[0][0] = newlayer
    learn.unfreeze()
    return learn

In [4]:
p = Config.data_path()/'freesound-audio-tagging-2019'; p
train_path = p/'train_noisy'
valid_path = p/'train_curated'

In [5]:
train = AudioList.from_csv(train_path, 'train_noisy.csv').split_none().label_from_df(cols='labels').train
valid = AudioList.from_csv(valid_path, 'train_curated.csv').split_none().label_from_df(cols='labels').train
print(train);print('\n'+str(valid))

LabelList (19815 items)
x: AudioList
AudioItem: 3.6281179138321997sec (160000 @ 44100hz).,AudioItem: 3.6281179138321997sec (160000 @ 44100hz).,AudioItem: 3.6281179138321997sec (160000 @ 44100hz).,AudioItem: 3.6281179138321997sec (160000 @ 44100hz).,AudioItem: 3.6281179138321997sec (160000 @ 44100hz).
y: CategoryList
Bathtub_(filling_or_washing),Motorcycle,Marimba_and_xylophone,Glockenspiel,Water_tap_and_faucet,Sink_(filling_or_washing),Raindrop
Path: /home/h/.fastai/data/freesound-audio-tagging-2019/train_noisy

LabelList (4970 items)
x: AudioList
AudioItem: 3.6281179138321997sec (160000 @ 44100hz).,AudioItem: 3.6281179138321997sec (160000 @ 44100hz).,AudioItem: 3.6281179138321997sec (160000 @ 44100hz).,AudioItem: 3.6281179138321997sec (160000 @ 44100hz).,AudioItem: 3.6281179138321997sec (160000 @ 44100hz).
y: CategoryList
Bark,Raindrop,Finger_snapping,Run,Finger_snapping
Path: /home/h/.fastai/data/freesound-audio-tagging-2019/train_curated


In [ ]:
max_length = int(10*44100) # seconds * sampling rate
bs = 64
tfm_params = {
    'max_to_pad': max_length,
    'use_spectro': True, 
    'cache_spectro': True, 
    'to_db_scale': True,
    'f_max': 250,
    'n_fft': 2206,
    'hop': 20,
    'n_mels': 2206
}

train.x.tfm_args = tfm_params
valid.x.tfm_args = tfm_params


ad, c= valid[6]
print(ad.shape)
ad.show()
print(c)
ad.duration

In [82]:

train = train.transform([tfm_sg_aug, tfm_sg_roll])
db = DataBunch.create(train, valid, bs=bs)
db

DataBunch;

Train: LabelList (19815 items)
x: AudioList
AudioItem: 10.0sec (441000 @ 44100hz).,AudioItem: 10.0sec (441000 @ 44100hz).,AudioItem: 10.0sec (441000 @ 44100hz).,AudioItem: 10.0sec (441000 @ 44100hz).,AudioItem: 10.0sec (441000 @ 44100hz).
y: CategoryList
Bathtub_(filling_or_washing),Motorcycle,Marimba_and_xylophone,Glockenspiel,Water_tap_and_faucet,Sink_(filling_or_washing),Raindrop
Path: /home/h/.fastai/data/freesound-audio-tagging-2019/train_noisy;

Valid: LabelList (4970 items)
x: AudioList
AudioItem: 10.0sec (441000 @ 44100hz).,AudioItem: 10.0sec (441000 @ 44100hz).,AudioItem: 10.0sec (441000 @ 44100hz).,AudioItem: 10.0sec (441000 @ 44100hz).,AudioItem: 10.0sec (441000 @ 44100hz).
y: CategoryList
Bark,Raindrop,Finger_snapping,Run,Finger_snapping
Path: /home/h/.fastai/data/freesound-audio-tagging-2019/train_curated;

Test: None

In [ ]:
print(db.train_ds[0][0].shape)
db.train_ds.get(1).show()
print(db.train_ds[0][1])

In [ ]:
learn = make_learner(db)

In [ ]:
learn.lr_find()
learn.recorder.plot()